In [46]:
import numpy as np
import math

### Read Input


In [6]:
incom = np.loadtxt('14_input.txt',delimiter="\n", comments=None,dtype=str)

incom[:5]

array(['118 ORE => 7 GTPZ', '6 RNQJN, 4 NQKVW => 4 DTQRC',
       '2 GBXJL => 3 XHBR', '4 BPZM => 9 LVDRH', '131 ORE => 3 RHBL'],
      dtype='<U98')

#### digest input into chemical ins and outs

In [7]:
incom[0].split(' => ')

['118 ORE', '7 GTPZ']

In [8]:
incom_split = [i.split(' => ') for i in incom]

In [9]:
incom_pies = [i[0].split(', ') +[i[1]] for i in incom_split]

In [10]:
incom_pies[:5]

[['118 ORE', '7 GTPZ'],
 ['6 RNQJN', '4 NQKVW', '4 DTQRC'],
 ['2 GBXJL', '3 XHBR'],
 ['4 BPZM', '9 LVDRH'],
 ['131 ORE', '3 RHBL']]

So, the first entries in a list are the inputs, the last entries are the chemicals generated.

Now, create a chemical class:

In [65]:
# Create a class

class Chem:
    def __init__(self,name):
        self.name = name
        self.recipies = []
        self.min_ore = [1,999999999] # [quantity, ore required]
        
        toe.append(self.name) # append self to table of elements
        
    def add_recipie(self, recipie):
        self.recipies.append(recipie)
        
    def min_ore_calc(self):
        ores = []
        for card in self.recipies:
            #quant_chem_gen = card[0][0]
            ore = 0
            for ingred in card[1:]:
                if ingred[1].min_ore[1] == 999999999:
                    ingred[1].min_ore_calc()
                ore += math.ceil(ingred[0]/ingred[1].min_ore[0])*ingred[1].min_ore[1]
            ores.append([card[0][0],ore])
        
        # Choose minimum ore:
        self.min_ore = ores[np.argmax([i[0]/i[1] for i in ores])]
        

In [66]:
toe = []

# Create ORE
ORE = Chem('ORE')
ORE.min_ore = [1,1]

for i in incom_pies:
    # Decompose entry into amount and ingredient
    amountp,ingredp = int(i[-1].split(' ')[0]), i[-1].split(' ')[1]
    if ingredp not in toe:
        vars()[ingredp] = Chem(ingredp)
    
    # Create recipie card
    recipie = []
    # first, add product to recipie
    recipie.append([amountp,vars()[ingredp]])
    
    for entry in i[:-1]:
        # Decompose entry into amount and ingredient
        amount,ingred = int(entry.split(' ')[0]), entry.split(' ')[1]
        if ingred not in toe: # add the ingredient to the toe if its not already there
            vars()[ingred] = Chem(ingred)
        # add ingredient to recipie
        recipie.append([amount,vars()[ingred]])
        
    # put recipie card into class
    vars()[ingredp].add_recipie(recipie)


In [67]:
toe[:5]

['ORE', 'GTPZ', 'DTQRC', 'RNQJN', 'NQKVW']

In [68]:
ORE.min_ore
    

[1, 1]

In [71]:
GBXJL.min_ore_calc()
GBXJL.min_ore

[8, 131]

In [48]:
multiple = 3
needed = 18

math.ceil(needed/multiple)

6

In [1]:
# Just in case I need an LCM:
def gcd(a,b):
    """Compute the greatest common divisor of a and b"""
    while b > 0:
        a, b = b, a % b
    return a
    
def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    return int(a * b / gcd(a, b))

In [20]:
incom_pies[1][0]#.split(' ')[0]

'6 RNQJN'

In [36]:
a = np.empty((2))

In [37]:
a

array([-3.10503618e+231, -2.68679134e+154])

In [40]:
np.append(a,a, axis=0)

array([-3.10503618e+231, -2.68679134e+154, -3.10503618e+231,
       -2.68679134e+154])

In [41]:
a

array([-3.10503618e+231, -2.68679134e+154])

In [52]:
a = np.array([[1,1,1],[2,1,3]]).T

In [53]:
a

array([[1, 2],
       [1, 1],
       [1, 3]])